## ETL Project for Top 10 Largest Banks by Market Capitalization

In [1]:
!pip install pandas
!pip install numpy
!pip install bs4

In [2]:
# imported required packages
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

## Successfully downloaded exchange_rate.csv file

In [3]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"
file_name = "exchange_rate.csv"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Open the file in write-binary mode and save the content
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f"'{file_name}' has been downloaded successfully.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")


'exchange_rate.csv' has been downloaded successfully.


## Task 1: Logging function

In [4]:
import datetime

def log_progress(message):
    # Get the current timestamp
    time_stamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Format the log message
    log_message = f"{time_stamp} : {message}\n"

    # Append the log message to the file
    with open("code_log.txt", "a") as log_file:
        log_file.write(log_message)

# Example of logging based on the tasks
log_progress("Preliminaries complete. Initiating ETL process")
# Later in your ETL code, you would log other messages at appropriate places


## Task 2 : Extraction of data

In [5]:
def log_progress(message):
    time_stamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_message = f"{time_stamp} : {message}\n"
    with open("code_log.txt", "a") as log_file:
        log_file.write(log_message)

def extract():
    # URL of the webpage
    url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"

    # Send a request to fetch the HTML content
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table under the heading "By market capitalization"
    table = soup.find('table', {'class': 'wikitable'})  # Adjust the selector as necessary

    # Initialize a list to hold the rows of the DataFrame
    data = []

    # Loop through the rows of the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        cols = row.find_all('td')
        if cols:
            # Extract data, remove unwanted characters and convert to float if necessary
            rank = int(cols[0].text.strip())
            name = cols[1].text.strip()
            market_cap = float(cols[2].text.strip().replace('\n', ''))  # Remove new line
            data.append([rank, name, market_cap])

    # Create a DataFrame from the data
    df = pd.DataFrame(data, columns=['Rank', 'Bank Name', 'Market Cap'])

    # Log the completion of the extraction process
    log_progress("Data extraction complete. Initiating Transformation process")

    return df

# Function call for extract() and print the DataFrame
df_banks = extract()
print(df_banks)

   Rank                                Bank Name  Market Cap
0     1                           JPMorgan Chase      432.92
1     2                          Bank of America      231.52
2     3  Industrial and Commercial Bank of China      194.56
3     4               Agricultural Bank of China      160.68
4     5                                HDFC Bank      157.91
5     6                              Wells Fargo      155.87
6     7                        HSBC Holdings PLC      148.90
7     8                           Morgan Stanley      140.83
8     9                  China Construction Bank      139.82
9    10                            Bank of China      136.81


## Task 3 : Transformation of data

In [6]:
import pandas as pd
import numpy as np

def log_progress(message):
    time_stamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_message = f"{time_stamp} : {message}\n"
    with open("code_log.txt", "a") as log_file:
        log_file.write(log_message)

def transform(df):
    # Read the exchange rate CSV file into a DataFrame
    exchange_rates = pd.read_csv('/content/exchange_rate.csv')

    # Convert the contents to a dictionary
    exchange_rate_dict = dict(zip(exchange_rates.iloc[:, 0], exchange_rates.iloc[:, 1]))

    # Adding new columns to the DataFrame based on exchange rates
    df['MC_GBP_Billion'] = [np.round(x * exchange_rate_dict['GBP'], 2) for x in df['Market Cap']]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate_dict['EUR'], 2) for x in df['Market Cap']]
    df['MC_INR_Billion'] = [np.round(x * exchange_rate_dict['INR'], 2) for x in df['Market Cap']]

    # Log the completion of the transformation process
    log_progress("Data transformation complete. Initiating Loading process")

    return df

# Function call for transform() and print the DataFrame
df_transformed = transform(df_banks)  # assuming df_banks is the DataFrame from the extract function
# print(df_transformed)  # Commented out as per the instructions


In [7]:
df_transformed

,Rank,Bank Name,Market Cap,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,1,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,2,Bank of America,231.52,185.22,215.31,19204.58
2,3,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,4,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,5,HDFC Bank,157.91,126.33,146.86,13098.63
5,6,Wells Fargo,155.87,124.70,144.96,12929.42
6,7,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
7,8,Morgan Stanley,140.83,112.66,130.97,11681.85
8,9,China Construction Bank,139.82,111.86,130.03,11598.07
9,10,Bank of China,136.81,109.45,127.23,11348.39


## Task 4: Loading to CSV

In [8]:
import pandas as pd
import datetime

# Define the log function
def log_progress(message):
    with open("code_log.txt", "a") as log_file:
        time_stamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_file.write(f"{time_stamp} : {message}\n")

# Define the function to load data to a CSV file
def load_to_csv(df, file_path):
    # Save DataFrame to CSV
    df.to_csv(file_path, index=False)
    # Log progress
    log_progress("Data saved to CSV file")

# Example usage
file_path = '/content/transformed_data.csv'  # You can replace this with the required path
load_to_csv(df_transformed, file_path)  # Assuming df_transformed is your DataFrame from transform()


In [9]:
print(f"CSV file saved at: {file_path}")


CSV file saved at: /content/transformed_data.csv


## Task 5: Loading to Database

In [10]:
import sqlite3
import pandas as pd
import datetime

# Define the log function
def log_progress(message):
    with open("code_log.txt", "a") as log_file:
        time_stamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_file.write(f"{time_stamp} : {message}\n")

# Define the function to load data to a database
def load_to_db(df, db_connection, table_name):
    # Load the DataFrame to the database table
    df.to_sql(table_name, db_connection, if_exists='replace', index=False)
    # Log progress
    log_progress("Data loaded to Database as a table, Executing queries")

# Example usage
db_name = 'Banks.db'
table_name = 'Largest_banks'

# Initiate SQLite connection
connection = sqlite3.connect(db_name)
log_progress("SQL Connection initiated")

# Call the load_to_db function
load_to_db(df_transformed, connection, table_name)  # Assuming df_transformed is your transformed DataFrame

# Close the connection
connection.close()
log_progress("Server Connection closed")


## Task 6: Function to Run queries on Database

In [11]:
import sqlite3
import datetime

# Define the log function
def log_progress(message):
    with open("code_log.txt", "a") as log_file:
        time_stamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_file.write(f"{time_stamp} : {message}\n")

# Define the function to execute queries
def run_queries(query, db_connection):
    print(f"Executing Query: {query}")
    cursor = db_connection.cursor()
    cursor.execute(query)

    # Fetch and print results for SELECT queries
    results = cursor.fetchall()
    for row in results:
        print(row)

    # Log progress
    log_progress("Process Complete")

# Example usage
db_name = 'Banks.db'
connection = sqlite3.connect(db_name)
log_progress("SQL Connection initiated")

# 1. Print the contents of the entire table
print("\nContents of the Entire Table:")
run_queries("SELECT * FROM Largest_banks", connection)

# 2. Print the average market capitalization of all the banks in Billion USD
print("\nAverage Market Capitalization in GBP:")
run_queries("SELECT AVG(MC_GBP_Billion) FROM Largest_banks", connection)

# 3. Print only the names of the top 5 banks
print("\nTop 5 Banks by Name:")
run_queries('SELECT "Bank Name" from Largest_banks LIMIT 5', connection)



Contents of the Entire Table:
Executing Query: SELECT * FROM Largest_banks
(1, 'JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71)
(2, 'Bank of America', 231.52, 185.22, 215.31, 19204.58)
(3, 'Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75)
(4, 'Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41)
(5, 'HDFC Bank', 157.91, 126.33, 146.86, 13098.63)
(6, 'Wells Fargo', 155.87, 124.7, 144.96, 12929.42)
(7, 'HSBC Holdings PLC', 148.9, 119.12, 138.48, 12351.26)
(8, 'Morgan Stanley', 140.83, 112.66, 130.97, 11681.85)
(9, 'China Construction Bank', 139.82, 111.86, 130.03, 11598.07)
(10, 'Bank of China', 136.81, 109.45, 127.23, 11348.39)

Average Market Capitalization in GBP:
Executing Query: SELECT AVG(MC_GBP_Billion) FROM Largest_banks
(151.98700000000002,)

Top 5 Banks by Name:
Executing Query: SELECT "Bank Name" from Largest_banks LIMIT 5
('JPMorgan Chase',)
('Bank of America',)
('Industrial and Commercial Bank of China',)
('Agricultural Bank of China'

In [12]:
# Define function to check column names
def check_columns(db_connection):
    cursor = db_connection.cursor()
    cursor.execute("PRAGMA table_info(Largest_banks)")
    columns = cursor.fetchall()
    print("Column Names in Largest_banks Table:")
    for col in columns:
        print(col)

# Check the column names in the Largest_banks table
check_columns(connection)


Column Names in Largest_banks Table:
(0, 'Rank', 'INTEGER', 0, None, 0)
(1, 'Bank Name', 'TEXT', 0, None, 0)
(2, 'Market Cap', 'REAL', 0, None, 0)
(3, 'MC_GBP_Billion', 'REAL', 0, None, 0)
(4, 'MC_EUR_Billion', 'REAL', 0, None, 0)
(5, 'MC_INR_Billion', 'REAL', 0, None, 0)


In [13]:
# Close the connection
connection.close()
log_progress("Server Connection closed")


## Task 7: Verify log entries

In [14]:
# Define a function to read the log file
def read_log_file(file_path):
    try:
        with open(file_path, 'r') as log_file:
            log_contents = log_file.readlines()
            for line in log_contents:
                print(line.strip())
    except FileNotFoundError:
        print("The log file does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
log_file_path = "code_log.txt"  # Replace with the correct path if necessary
read_log_file(log_file_path)


2024-10-16 11:59:02 : Preliminaries complete. Initiating ETL process
2024-10-16 11:59:04 : Data extraction complete. Initiating Transformation process
2024-10-16 11:59:04 : Data transformation complete. Initiating Loading process
2024-10-16 11:59:04 : Data saved to CSV file
2024-10-16 11:59:04 : SQL Connection initiated
2024-10-16 11:59:04 : Data loaded to Database as a table, Executing queries
2024-10-16 11:59:04 : Server Connection closed
2024-10-16 11:59:04 : SQL Connection initiated
2024-10-16 11:59:04 : Process Complete
2024-10-16 11:59:04 : Process Complete
2024-10-16 11:59:04 : Process Complete
2024-10-16 11:59:04 : Server Connection closed
2024-10-16 12:08:59 : Preliminaries complete. Initiating ETL process
2024-10-16 12:09:00 : Data extraction complete. Initiating Transformation process
2024-10-16 12:09:00 : Data transformation complete. Initiating Loading process
2024-10-16 12:09:01 : Data saved to CSV file
2024-10-16 12:09:01 : SQL Connection initiated
2024-10-16 12:09:01 : 